#### 1. 二叉树节点类

In [40]:
# -*- coding: UTF-8 -*-
class BinNode(object):
    def __init__(self, data, parent=None, lc=None, rc=None, height=0,):
        self.data = data
        self.parent = parent
        self.height = height
        self.lc = lc
        self.rc = rc

    def __eq__(self, other):
        return self.data == other.data

    def __lt__(self, other):
        return self.data < other.data
    
    def insert_aslchild(self,data):
        new_node = BinNode(data,self)
        self.lc = new_node
        return new_node
    
    def insert_asrchild(self,data):
        new_node = BinNode(data,self)
        self.rc = new_node
        return new_node

if __name__ == "__main__":
    n = BinNode(2)
    k = BinNode(2)

#### 2. 二叉树节点工具类
将判断二叉节点状态,性质. 定位与之相关的叔叔,兄弟节点等逻辑封装

In [60]:
class BinNodeUtil(object):
    @staticmethod
    def is_root(node):
        return not node.parent
    @staticmethod
    def is_lchild(node):
        return (not BinNodeUtil.is_root(node))&(node.parent.lc is node)
    @staticmethod
    def is_rchild(node):
        return (not BinNodeUtil.is_root(node))&(node.parent.rc is node)
    @staticmethod
    def has_parent(node):
        return not BinNodeUtil.is_root(node)
    @staticmethod
    def has_lchild(node):
        return not node.lc is None
    @staticmethod
    def has_rchild(node):
        return not node.rc is None
    @staticmethod
    def has_child(node):
        return BinNodeUtil.has_lchild(node) or BinNodeUtil.has_rchild(node)
    @staticmethod
    def has_bothchild(node):
        return BinNodeUtil.has_lchild(node) & BinNodeUtil.has_rchild(node)
    @staticmethod
    def is_leaf(node):
        return not BinNodeUtil.has_child(node)
    @staticmethod
    def sibling(node):
        try:
            return  node.parent.rc if BinNodeUtil.is_lchild(node) else node.parent.lc
        except  AttributeError:
            return None
    @staticmethod
    def uncile(node):
        try:
            parent = node.parent
            return parent.parent.rc if BinNodeUtil.is_lchild(parent) else parent.parent.lc
        except  AttributeError:
            return None
    @staticmethod
    def from_parent_to(node,value=None):
        '''来自父亲的引用'''
        if (BinNodeUtil.is_lchild(node)):
            node.parent.lc = None
        if (BinNodeUtil.is_rchild(node)):
            node.parent.rc = None
    

#### 3. 二叉树类
1. 二叉树类包含二叉树的常用操作  
  1. 更新树中, 每个节点的高度  
    1. 二叉树任意节点的高度 = 其孩子节点的最大高度+1  `update_height`  
    2. 一旦有节点插入或删除, 要更新其所有祖先的高度  `update_height_above`   
  2. 插入结点到二叉树中, 3种方式  :  
    1. 插入根节点`insert_root`    
    2. 插入树中某个节点的左孩子`insert_as_lc`  
    3. 插入树中某个节点的右孩子`insert_as_rc`
  3. 子树接入  
   任意二叉树, 都可以作为另一个二叉树的左子树/右子树插入`attach_as_lc`
  4. 子树删除  
   子树删除要删除该节点和节点的所有孩子节点, 并更新树的size和height


In [61]:
class BinTree(object):
    def __init__(self):
        self.size = 0
        self.root = None
        
    def empty(self):
        return self.root is None
    
    def __eq__(self,other):
        return (self.root is not None) and (other.root is not None) and (self.root is other.root)
    
    def stature(self,node):
        return -1 if node is None else node.height
    
    def update_height(self,node):
        node.height = 1 + max(self.stature(node.lc), self.stature(node.rc))
        return node.height
    
    def update_height_above(self,node):
        while node is not None:
            self.update_height(node)
            node = node.parent
            
    def insert_root(self,data):
        self.size = 1
        self.root = BinNode(data)
        return self.root
    
    def insert_as_lc(self, node,data):
        self.size = self.size + 1
        lc = node.insert_aslchild(data)
        self.update_height_above(node)
        return lc
    
    def insert_as_rc(self, node , data):
        self.size = self.size + 1
        rc = node.insert_asrchild(data)
        self.update_height_above(node)
        return rc
    
    def attach_as_lc(self, node,other_tree):
        '''node: 该二叉树中的节点
           other_tree : 作为子树接入该二叉树. 接入点为node节点的左孩子'''
        node.lc = other_tree.root
        other_tree.root.parent = node
        self.size = self.size + other_tree.size
        self.update_height_above(node)
        return node
    
    def attach_as_rc(self, node,other_tree):
        '''node: 该二叉树中的节点
           other_tree : 作为子树接入该二叉树. 接入点为node节点的右孩子'''
        node.rc = other_tree.root
        other_tree.root.parent = node
        self.size = self.size + other_tree.size
        self.update_height_above(node)
        return node
    
    def remove(self,node):
        '''删除该二叉树中节点node及其后代'''
        n = self.removeAt(node)
        
        BinNodeUtil.from_parent_to(node,None)
        self.update_height_above(node.parent)
        self.size = self.size - n
        return n
        
    def removeAt(self,node):
        '''删除节点node及其后代'''
        if node is None:
            return 0
        n = 1 + self.removeAt(node.lc) + self.removeAt(node.rc)
        node = None 
        return n
    

#### 4. 二叉树工具类   
2. 遍历 , 树形如下 : 
  <img src='img/bintreeinR.png' width='50%' height='50%'>
   1. 递归版中序遍历`travIn_R` 

In [70]:
class  BinTreeUtility(object):
    @staticmethod
    def travIn_R(node , visit):
        if node is None : 
            return 
        BinTreeUtility.travIn_R(node.lc , visit)
        visit (node.data)
        BinTreeUtility.travIn_R(node.rc , visit)

In [76]:
if __name__ == "__main__":
    # 构建树
    tree = BinTree()
    i = tree.insert_root('i')
    d =  tree.insert_as_lc(i,'d')
    l =  tree.insert_as_rc(i,'l')
    c = tree.insert_as_lc(d,'c')
    h = tree.insert_as_rc(d,'h')
    a = tree.insert_as_lc(c,'a')
    b = tree.insert_as_rc(a,'b')
    f = tree.insert_as_lc(h,'f')
    e =tree.insert_as_lc(f,'e')
    g = tree.insert_as_rc(f,'g')
    k = tree.insert_as_lc(l,'k')
    n = tree.insert_as_rc(l,'n')
    j = tree.insert_as_lc(k,'j')
    m = tree.insert_as_lc(n,'m')
    p = tree.insert_as_rc(n,'p')
    o = tree.insert_as_lc(p,'o')
   
    orders = []
    def visit(x):
        orders.append(x)
    # 遍历树
    BinTreeUtility.travIn_R(tree.root,visit)
    print orders

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
